# 先開啓我們的範例 Excel 檔案

```python
import xlwings as xw
# 打開你的 stock_price_data.xlsx 檔案
wb = xw.Book(r"範例檔案 stock_price_data.xlsx 的路徑")

tsmc_sheet = wb.sheets["2330"]

# 算出所有的報酬率
for i in range(3, 97):
    daily_return = (tsmc_sheet.cells(i, 2).value - tsmc_sheet.cells(i-1, 2).value) / tsmc_sheet.cells(i-1, 2).value
    tsmc_sheet.cells(i, 3).value = daily_return
```

# 回顧上一堂課...

要如何計算 `2017/10/11` 至 `2018/2/26` 的三日簡單移動平均？

```python
for i in range(4, 97):
    today = tsmc_sheet.cells(i, ____).value
    yesterday = tsmc_sheet.cells(i-1, ____).value
    day_b4_yesterday = ______________________
    daily_return = (today + yesterday + day_b4_yesterday) / 3
    tsmc_sheet.cells(i, ____).value = daily_return
```

# .formula 屬性


每一個儲存格都有一個 formula 屬性，可以利用 python 動態設定該儲存格的公式


**注意公式需要用字串 "" 表示，都是從 "=" 開始**

```python
tsmc_sheet.cells(4, "D").formula = "=AVERAGE(B2:B4)"
```

當然，我也可以用 range() 來選擇 D4:

```python
tsmc_sheet.range('D4').formula = "=AVERAGE(B2:B4)"
```



# 隨堂練習

利用設定 .formula 屬性的方式計算每一天的三日移動平均：

```python
for i in range(4, 97):
    formula = ________________________
    tsmc_sheet.cells(i, ____).formula = formula
```

In [0]:
for i in range(4, 97):
    formula = "=AVERAGE(B{}:B{})".format(i-2, i)
    print(formula)
    tsmc_sheet.cells(i, "D").formula = formula

# .color 屬性

每一個儲存格都有一個 color 屬性，可以利用 python **動態設定儲存格的底色**

注意 color 的表達方式是將三原色 (RGB 值) 用 tuple (元組) 來表示：

```python
# 紅色亮度調最大，藍色、綠色調最小
(255, 0, 0)
```

所以若要把 A1 儲存格的底色換成紅色：

```python
tsmc_sheet.cells(1, "A").color = (255, 0, 0)
```

# 設定字體顔色（僅限 windows）

# .font.ColorIndex 屬性

在 VBA 語言透過指定顔色的編號設定儲存格的底色

```vb
Range("A1").Font.colorIndex = 3
```

而透過 xlwings 可以使用：

```python
tsmc_sheet.cells(1, "A").api.Font.colorIndex = 3
```

## 一些常用的顔色編號：

![](https://www.dropbox.com/s/c8prdhqb0epnst4/2.PNG?dl=1)


[關於顔色的官方文件](https://docs.microsoft.com/en-us/office/vba/api/excel.colorindex)

# 隨堂練習：

將之前計算出來的報酬率加上一點視覺效果:

若當天**上漲，就將儲存格底色變成紅色**

**若當天下跌，就將儲存格底色變成綠色**

## 提示影片：

[![IMAGE ALT TEXT](http://img.youtube.com/vi/xDztsNqNaq4/0.jpg)](http://www.youtube.com/watch?v=xDztsNqNaq4 "Video Title")

# 動態偵測範圍/資料筆數

在上一堂課，我們利用 for loop 計算了所有的報酬率

In [0]:
# 算出所有的報酬率
for i in range(3, 97):
    daily_return = (tsmc_sheet.cells(i, 2).value - tsmc_sheet.cells(i-1, 2).value) / tsmc_sheet.cells(i-1, 2).value
    tsmc_sheet.cells(i, 3).value = daily_return

問題來了，要是試算表上**多了或少了一筆資料，原本的程式就爆炸了...**

**若你寫出了一個需要頻繁修改的程式，那就失去了自動化的意義**

我們需要一個能夠動態偵測我們有幾筆資料需要計算的方法

在 VBA 的世界裡，有個好用的函數叫做 **End()**，可以動態偵測一從一個儲存格開始，**連續、有值範圍的最後一個儲存格**：


|  **常數**       | **方向**          |
| ------------- |:-----------:| 
| **End(xlDown)**    | ↓      | 
| **End(xlUp)**     | ↑      | 
| **End(xlRight)** | →      |
| **End(xlRight)** | ←      |




範例1：會從 A1 儲存格開始，**往下**查找到最後一個有值的儲存格
```vb
last_row = Range("A1").End(xlDown).Row
```

範例2：會從 A1 儲存格開始，**往右**查找到最後一個有值的儲存格
```vb
last_col = Range("A1").End(xlToRight).Column
```

幸運的是，`xlwings` 也具備了一模一樣的功能：

In [0]:
from xlwings.constants import Direction

last_cell = tsmc_sheet.cells(1, "A").end(Direction.xlDown)
last_cell.color = (255, 0, 0)

In [0]:
# 會 VBA 的人，馬上可以看出 xlwings 原汁原味的實作了 Python 版的 End() 函數
from xlwings.constants import Direction

# 從 A1 儲存格開始，往下查找到最後一個有值的儲存格
last_cell = tsmc_sheet.range("A1").end(Direction.xlToRight)
# 把從 A1 開始，最右下角的儲存格底色換成紅色
last_cell.color = (255, 0, 0)

In [0]:
from xlwings.constants import Direction
# 查找從 A1 開始、連續的、有值的範圍最右下角的儲存格，將其底色標為紅色
tsmc_sheet.range("A1").end(Direction.xlDown).end(Direction.xlToRight).color = (255, 0, 0)
# 將該儲存格存入 last_cell 變數
last_cell = tsmc_sheet.range("A1").end(Direction.xlDown).end(Direction.xlToRight)
# 使用 .row 屬性取得 row 值，也就是我們的資料有幾行
last_cell.row

In [0]:
# 另一種比較簡潔的寫法則是...
last_cell = tsmc_sheet.range("B4").end("down")
print(last_cell)
# 會找到 B96
last_cell = tsmc_sheet.range("B4").end("up")
print(last_cell)
# 會找到 B1
last_cell = tsmc_sheet.range("B4").end("left")
print(last_cell)
# 會找到 A4
last_cell = tsmc_sheet.range("A1").end("right")
print(last_cell)
# 會找到 E1

# 隨堂練習

利用動態偵測最後一個 row 的方式計算臺積電所有的報酬率、並將結果寫入 Excel

```python
from xlwings.constants import Direction

last_row = tsmc_sheet.range("B1").end(____________).row

for i in range(3, _________):
    daily_return = _________________________________
    ______________________ = daily_return
```